### Adaboost Algorithm 

#### 1. Initialize weights $w_{i}=(1/N)$ (Assign each training point equal weights.)
#### 2. Calculate error rate for each $'h'$ (classifier) 
$\mathcal{E} = \sum_{wrong} w_{i}$
#### 3. Pick "best" h (classifier) with smallest error Or choose classifier which has error rate furthest from $1/2$ i.e.
$max |\mathcal{E} - \frac{1}{2}|$
#### 4. Calculate votting power for 'h'
$\alpha = \frac{1}{2} ln \frac{1-\mathcal{E}}{\mathcal{E}}$
#### 5. Are we finished ? 
Is 'H' (Overall classifier) is good enough ?
Enough Rounds ?
No good classifier left ? (best 'h' has $\mathcal{E} = 1/2$)
#### 6. If No: 
Update weights to emphasize points that were misclassified.
$w_{new} = \begin{cases} \frac{1}{2}\frac{1}{1-\mathcal{E}}w_{old} \quad \text{if right} \\  \frac{1}{2}\frac{1}{\mathcal{E}}w_{old} \quad \text{if wrong}\end{cases}$
</br>Go to step 2 and repeat.
#### 7. If Yes:
The Overall classsifier is 'H(x)'

In [2]:
import numpy as np
import math

In [35]:
class Classifiers:
    """dummy classifier which can be replaced with actual classifiers."""
    
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.Y_predict = []
        
    def classifier_one(self):
        labels = []
        for each_data_point in X:
            if each_data_point[0] < 2:
                labels.append(1)
            else:
                labels.append(-1)
        return labels
    
    def classifier_one_predict(self, x):
        if x[0] < 2:
            return 1 
        else:
            return -1
    
    def classifier_two(self):
        labels = []
        for each_data_point in X:
            if each_data_point[0] < 4:
                labels.append(1)
            else:
                labels.append(-1)
        return labels
    
    def classifier_two_predict(self, x):
        if x[0] < 4:
            return 1 
        else:
            return -1
    
    def classifier_three(self):
        labels = []
        for each_data_point in X:
            if each_data_point[0] < 6:
                labels.append(1)
            else:
                labels.append(-1)
        return labels
    
    def classifier_three_predict(self, x):
        if x[0] < 6:
            return 1 
        else:
            return -1
    
    def classifier_four(self):
        labels = []
        for each_data_point in X:
            if each_data_point[0] > 2:
                labels.append(1)
            else:
                labels.append(-1)
        return labels 
    
    def classifier_four_predict(self, x):
        if x[0] > 2:
            return 1 
        else:
            return -1
    
    def classifier_five(self):
        labels = []
        for each_data_point in X:
            if each_data_point[0] > 4:
                labels.append(1)
            else:
                labels.append(-1)
        return labels
    
    def classifier_five_predict(self, x):
        if x[0] > 4:
            return 1 
        else:
            return -1
    
    def classifier_six(self):
        labels = []
        for each_data_point in X:
            if each_data_point[0] > 6:
                labels.append(1)
            else:
                labels.append(-1)
        return labels 
    
    def classifier_six_predict(self, x):
        if x[0] > 6:
            return 1 
        else:
            return -1
    
    def misclassified_points(self, Y_predict):
        Y = np.array(self.Y)
        Y_predict = np.array(Y_predict)
        #import pdb; pdb.set_trace()
        index_ = np.where(Y != Y_predict)
        return X[index_], index_
    
    def classify_using_all(self):
        classifiers = [('classifier_one', self.classifier_one()),
                       ('classifier_two', self.classifier_two()),
                       ('classifier_three', self.classifier_three()),
                       ('classifier_four', self.classifier_four()),
                       ('classifier_five', self.classifier_five()),
                       ('classifier_six', self.classifier_six())]
        classifier_misclassified_point = []
        classifier_misclassified_pt_ind = []
        classifier_index = 0 
        for name, each_classifier in classifiers:
            labels = each_classifier
            points, index_ = self.misclassified_points(labels)
            classifier_misclassified_point.append(points)
            classifier_misclassified_pt_ind.extend(index_)
        return np.array(classifier_misclassified_point), classifier_misclassified_pt_ind
    
# data_set  
X = np.array([[1, 5], [5, 5], [3, 3], [1, 1], [5,1]])
Y = np.array([1, 1, -1, 1, 1])


class Adaboost_imp(Classifiers):
    """Adaboost algo"""
    def __init__(self, X, Y, max_iter=5):
        super().__init__(X, Y)
        self.X = X
        self.Y = Y 
        self.classifiers_mis_points, self.classifier_misclassified_pt_ind = self.classify_using_all()
        self.error_rate = np.zeros(self.classifiers_mis_points.shape)
        self.weights = np.zeros((len(self.X), ))
        #self.weight_new = np.zeros((len(self.X), 1))
        self.classifiers_count = len(self.classifiers_mis_points)
        self.best_classifier = None
        self.max_iter = max_iter
        self.alphas = [0]*self.classifiers_count
        self.classifiers = {0: self.classifier_one_predict, 1: self.classifier_two_predict,
                           2: self.classifier_three_predict, 3: self.classifier_four_predict,
                           4: self.classifier_five_predict}
        
    def overall_classifier(self, x):
        total = 0
        for clas, each_alpha in enumerate(self.alphas):
            if each_alpha:
                total += each_alpha * self.classifiers[clas](x)
        return np.sign(total)
    
    def initialize_weight(self):
        self.weights[:] = 1 / len(self.X) 
        
        print("initial weights %s" %self.weights)
    
    def error_rate_(self):
        
        for each_clas in range(self.classifiers_count):
            self.error_rate[each_clas] = sum(self.weights[self.classifier_misclassified_pt_ind[each_clas]])
        
        print("error_rate %s" %self.error_rate)
    
    def pick_best_classifier(self, based_on_smallest_error=False):
        if based_on_smallest_error:
            self.best_classifier = np.argsort(self.error_rate)[0]
        # todo self.classifier_misclassified_pt_ind
    
    def votting_power(self):
        self.alphas[self.best_classifier] = 0.5 * math.log((1 - self.error_rate[self.best_classifier]) / self.error_rate[self.best_classifier])
        
    
    def is_finished(self):
        if self.error_rate[self.best_classifier] == 0.5:
            return True
        return False
    
    def update_weights(self):
        weight_new = [0]*len(self.weights)
        # misclassified pts 
        for pt_ind, pt in enumerate(self.X):
            if pt_ind not in self.classifier_misclassified_pt_ind[self.best_classifier]:
                weight_new[pt_ind] = 0.5 * (1/ (1 - self.error_rate[self.best_classifier])) * self.weights[pt_ind]
            else:
                weight_new[pt_ind] = 0.5 * (1/ (self.error_rate[self.best_classifier])) * self.weights[pt_ind]
        
        self.weights = np.array(weight_new)
        print("weight new %s" %self.weights)
        
    def algo(self):
        self.initialize_weight()
        for iter_ in range(self.max_iter):
            self.error_rate_()
            self.pick_best_classifier(based_on_smallest_error=True)
            self.votting_power()
            if self.is_finished(): break
            self.update_weights() 

    
ada_boost = Adaboost_imp(X, Y)

In [36]:
ada_boost.algo()



initial weights [0.2 0.2 0.2 0.2 0.2]
error_rate [0.4 0.6 0.2 0.6 0.4 0.8]
weight new [0.125 0.125 0.5   0.125 0.125]
error_rate [0.25 0.75 0.5  0.75 0.25 0.5 ]
weight new [0.08333333 0.25       0.33333333 0.08333333 0.25      ]
error_rate [0.5        0.83333333 0.33333333 0.5        0.16666667 0.66666667]
weight new [0.25 0.15 0.2  0.25 0.15]
error_rate [0.3 0.5 0.2 0.7 0.5 0.8]
weight new [0.15625 0.09375 0.5     0.15625 0.09375]
error_rate [0.1875 0.6875 0.5    0.8125 0.3125 0.5   ]
weight new [0.09615385 0.25       0.30769231 0.09615385 0.25      ]


In [38]:
ada_boost.overall_classifier([3, 3])

-1.0

## References 
1. Boosting (Adaboost) by Jessica Noss MIT [link](https://www.youtube.com/watch?v=gmok1h8wG-Q)